# Live cu-inj-live-impact 

In [3]:
# Setup directories, and convert dashboard notebook to a script for importing
!./setup.bash

[NbConvertApp] Converting notebook make_dashboard.ipynb to script
[NbConvertApp] Writing 3662 bytes to make_dashboard.py


In [1]:
from impact import evaluate_impact_with_distgen, run_impact_with_distgen
from impact.tools import isotime
from impact.evaluate import  default_impact_merit
from impact import Impact

from make_dashboard import make_dashboard

In [2]:
import pandas as pd

import json
import epics

import os
from time import sleep, time

# Nicer plotting
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

In [4]:
ROOT = os.getcwd()

# Configuration

In [5]:
PREFIX = 'cu-inj-live-impact'

# Directory for summary output
OUTPUTDIR = os.path.join(ROOT, 'output')

# Directory to output plots
PLOTDIR = os.path.join(ROOT, 'plot')

# Directory for archive files
APATH = os.path.join(ROOT, 'archive')


# PV -> Sim conversion table
CSV =  'pv_mapping/cu_inj_impact.csv'

# Base input files
CONFIG0 = {'impact_config': os.path.expandvars('$LCLS_LATTICE/impact/models/cu_inj/ImpactT.yaml'),
          'distgen_input_file': os.path.expandvars('$LCLS_LATTICE/distgen/models/cu_inj/distgen.yaml')
         }

# Base settings
SETTINGS0 = {
 'distgen:t_dist:length:value': 4 * 1.65 ,     #  Inferred pulse stacker FWHM: 4 ps, converted to tukey length
 'distgen:n_particle': 10000,   
 'stop': 16.5, 
 'timeout': 100000,
 'header:Nx': 32,
 'header:Ny': 32,
 'header:Nz': 32,
 #'total_charge':0, # Set to 0 to turn off Space Charge for faster execution 
 'change_timestep_1:dt':4e-12, 
 
 # Parallel control
 'header:Nprow':4,
 'header:Npcol':4,
 'use_mpi':True
   }

# EPICS -> Simulation settings

In [6]:
IRIS_DIAMETER_FROM_ENUM_IN_MM = {0:'LCLS', # LCLS cutout picture
       1 :1.8 ,
       2 :1.6 ,
       3 :1.4 ,
       4 :1.2 ,
       5 :1.1 ,
       6 :1.0 ,
       7 :0.8 ,
       8 :0.6 ,
       9 :0.5 ,
       10: 0.4 , 
       11: 0.3 }

def get_live_settings(csv, base_settings={}):
    """
    Fetches live settings for all devices in the CSV table, and translates them to simulation inputs
     
    """
    df = pd.read_csv(csv)
    pv_names = list(df['device_pv_name'])
    
    df['pv_value'] = epics.caget_many(pv_names)
    df.set_index('device_pv_name', inplace=True)
    
    # Custom for enum
    iris_enum = int(df.at['IRIS:LR20:130:CONFG_SEL', 'pv_value'])
    df.at['IRIS:LR20:130:CONFG_SEL', 'pv_value'] = IRIS_DIAMETER_FROM_ENUM_IN_MM[iris_enum]
    
    # Assign impact
    df['impact_value'] = df['impact_factor']*df['pv_value'] 
    
    # Collect settings
    settings = base_settings.copy()
    settings.update(dict(zip(df['impact_name'], df['impact_value'])))

    return settings

LIVE_SETTINGS = get_live_settings(CSV, SETTINGS0)
LIVE_SETTINGS

{'distgen:t_dist:length:value': 6.6,
 'distgen:n_particle': 10000,
 'stop': 16.5,
 'timeout': 100000,
 'header:Nx': 32,
 'header:Ny': 32,
 'header:Nz': 32,
 'change_timestep_1:dt': 4e-12,
 'header:Nprow': 4,
 'header:Npcol': 4,
 'use_mpi': True,
 'distgen:r_dist:sigma_xy:value': 0.39999996,
 'SOL1:solenoid_field_scale': 0.23848906168410197,
 'CQ01:b1_gradient': -0.0035053943200264097,
 'SQ01:b1_gradient': -0.001067800332906425,
 'L0A_phase:dtheta0_deg': 0.0,
 'L0B_phase:dtheta0_deg': -2.5,
 'L0A_scale:voltage': 58000000.0,
 'L0B_scale:voltage': 70959535.28927356,
 'QA01:b1_gradient': 3.0632439873091992,
 'QA02:b1_gradient': -2.3396812820821,
 'QE01:b1_gradient': 2.1838888890636,
 'QE02:b1_gradient': -0.30859259261728,
 'QE03:b1_gradient': -3.5556574076918603,
 'QE04:b1_gradient': 3.04366666691016}

In [7]:
#%%time
#I0 = run_impact_with_distgen(LIVE_SETTINGS, **CONFIG0, verbose=True )
    

In [8]:
#I0.plot(['sigma_x', 'sigma_y'], figsize=(16,9))

# Get live 

In [9]:
# Old routine
def make_plot(impact_object, itime=None, dir='/nfs/slac/g/beamphysics/cmayes/live-test/plot/'):
    fig = impact_object.plot(y=['norm_emit_x', 'norm_emit_y'], 
                             #y2=['mean_kinetic_energy'], 
                             y2=['sigma_x', 'sigma_y', 'sigma_z'],
                             return_figure=True, figsize=(16,9), include_labels=False, xlim=(0,20), ylim=(0, 2e-6))
    if not itime:
        itime=isotime()
        
    run_time = impact_object.output['run_info']['run_time']        
    title=f'{PREFIX}: Acquired settings at {itime}, simulation run time: {run_time/60:5.1f} min'
    fig.axes[0].set_title(title)        
    fname=f'{PREFIX}-{itime}.png'
    fname = os.path.join(dir, fname)
    plt.savefig(fname, dpi=150)
    plt.clf() # Don't show in the notebook
    
    print(title)
    print('Plot written to:', fname)
    return fname

#make_plot(I0, dir=PLOTDIR)

In [10]:
def my_merit(impact_object, itime):
    merit0 = default_impact_merit(impact_object)
    #plot_file = make_plot(impact_object, itime=itime, dir=PLOTDIR)
    plot_file = make_dashboard(impact_object, itime=itime, outpath=PLOTDIR)
    print('Dashboard written:', plot_file)
    plt.close('all')
    merit0['plot_file'] = plot_file
    merit0['isotime'] = itime
    return merit0

In [11]:
def run1():
    mysettings = get_live_settings(CSV, SETTINGS0)
    itime = isotime()
    res = evaluate_impact_with_distgen(mysettings,
                                       merit_f=lambda x: my_merit(x, itime),
                                       archive_path=APATH,
                                       **CONFIG0 )
    fname = fname=f'{OUTPUTDIR}/{PREFIX}-{itime}.json'
    json.dump(res, open(fname, 'w'))
    print('Written:', fname)
    return res
    

In [12]:
#run1()

# loop it

In [ ]:
while True:
    run1()